# MISTRAL OCR PDF to MD Translator

## 來源說明

本 Notebook 修改自 Mistral 官方範例：

- 原始文件說明：[https://docs.mistral.ai/capabilities/document/](https://docs.mistral.ai/capabilities/document/)
- 原始 Colab Notebook：[https://colab.research.google.com/github/mistralai/cookbook/blob/main/mistral/ocr/structured_ocr.ipynb](https://colab.research.google.com/github/mistralai/cookbook/blob/main/mistral/ocr/structured_ocr.ipynb)

> 本 Notebook 為個人學習與實驗用途所做修改，非官方版本。

經由 Claude 整理 mistralocr_pdf2md.ipynb 的版本

## PDF Mistral OCR 匯出工具

本 Notebook 可將 PDF 文件自動化轉換為 Markdown 格式，包含以下流程：

1. 使用 **Mistral OCR** 模型辨識 PDF 內文與圖片
2. 將辨識結果組成含圖片的 Markdown 檔
3. 使用 **Gemini** 模型將英文內容翻譯為**台灣繁體中文**
4. 匯出 Markdown 檔（原文版 + 翻譯版）與對應圖片

## 1. Import Dependencies & Setup
First, consolidate all imports at the beginning of the notebook:

In [ ]:
# Standard libraries
import os
import json
import base64
import time
from pathlib import Path

# Third-party libraries
from IPython.display import Markdown, display
from pydantic import BaseModel
from dotenv import load_dotenv

# Mistral AI
from mistralai import Mistral
from mistralai.models import OCRResponse, ImageURLChunk, DocumentURLChunk, TextChunk

# Google Gemini
from google import genai
from google.genai import types

## 2. Configuration & Environment Setup
Create a dedicated section for loading environment variables and initializing clients:

In [3]:
# Load environment variables
load_dotenv()

# Initialize Mistral client
mistral_api_key = os.getenv("MISTRAL_API_KEY")
if not mistral_api_key:
    raise ValueError("❌ 找不到 MISTRAL_API_KEY，請檢查 .env 是否正確設置。")
mistral_client = Mistral(api_key=mistral_api_key)

# Initialize Gemini client
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("❌ 未在 .env 找到 GEMINI_API_KEY，請確認已正確設置。")
gemini_client = genai.Client(api_key=gemini_api_key)

## 3. Define Utility Functions
Group all utility functions together:

In [4]:
# Pydantic models
class StructuredOCR(BaseModel):
    file_name: str
    topics: list[str]
    languages: str
    ocr_contents: dict

def retry_with_backoff(func, retries=5, base_delay=1.5):
    """Retry a function with exponential backoff."""
    for attempt in range(retries):
        try:
            return func()
        except Exception as e:
            if "429" in str(e):
                wait_time = base_delay * (2 ** attempt)
                print(f"⚠️ API rate limit hit. Retrying in {wait_time:.1f}s...")
                time.sleep(wait_time)
            else:
                raise e
    raise RuntimeError("❌ Failed after multiple retries.")

def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:
    """Replace image placeholders in markdown with base64-encoded images."""
    for img_name, base64_str in images_dict.items():
        markdown_str = markdown_str.replace(
            f"![{img_name}]({img_name})", f"![{img_name}]({base64_str})"
        )
    return markdown_str

def get_combined_markdown(ocr_response: OCRResponse) -> str:
    """Combine OCR text and images into a single markdown document."""
    markdowns: list[str] = []
    for page in ocr_response.pages:
        image_data = {img.id: img.image_base64 for img in page.images}
        markdowns.append(replace_images_in_markdown(page.markdown, image_data))
    return "\n\n".join(markdowns)

def insert_ocr_below_images(markdown_str, ocr_img_map, page_idx):
    """Insert OCR results below images in markdown."""
    for img_id, ocr_text in ocr_img_map.get(page_idx, {}).items():
        markdown_str = markdown_str.replace(
            f"![{img_id}]({img_id})",
            f"![{img_id}]({img_id})\n\n> 📄 Image OCR Result：\n\n```json\n{ocr_text}\n```"
        )
    return markdown_str

def save_images_and_replace_links(markdown_str, images_dict, page_idx, image_folder="images"):
    """Save base64 images to files and update markdown links."""
    os.makedirs(image_folder, exist_ok=True)
    image_id_to_path = {}

    for i, (img_id, base64_str) in enumerate(images_dict.items()):
        img_bytes = base64.b64decode(base64_str.split(",")[-1])
        img_path = f"{image_folder}/page_{page_idx+1}_img_{i+1}.png"
        with open(img_path, "wb") as f:
            f.write(img_bytes)
        image_id_to_path[img_id] = img_path

    for img_id, img_path in image_id_to_path.items():
        markdown_str = markdown_str.replace(
            f"![{img_id}]({img_id})", f"![{img_id}]({img_path})"
        )

    return markdown_str

## 4. PDF Processing Functions
Create dedicated functions for PDF processing:

In [5]:
def select_pdf_file():
    """Select a PDF file from the current directory."""
    pdf_dir = Path(".")
    pdf_files = sorted([f for f in pdf_dir.glob("*.pdf")])
    
    if not pdf_files:
        raise FileNotFoundError("❌ 沒有找到任何 PDF 檔案，請確認放在正確資料夾中。")
    
    print("📚 可用的 PDF 檔案：")
    for i, f in enumerate(pdf_files):
        print(f"  [{i+1}] {f.name}")
    
    choice = input("👉 請輸入要處理的檔案編號： ").strip()
    pdf_file = pdf_files[int(choice)-1]
    print(f"✅ 已選擇：{pdf_file}")
    
    return pdf_file

def process_pdf_with_mistral_ocr(pdf_file, client):
    """Process PDF with Mistral OCR."""
    # Upload to mistral
    uploaded_file = client.files.upload(
        file={
            "file_name": pdf_file.stem,
            "content": pdf_file.read_bytes(),
        },
        purpose="ocr"
    )
    
    signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)
    
    # OCR analyze PDF
    pdf_response = client.ocr.process(
        document=DocumentURLChunk(document_url=signed_url.url),
        model="mistral-ocr-latest",
        include_image_base64=True
    )
    
    return pdf_response

def process_images_with_ocr(pdf_response, mistral_client):
    """Process images from PDF pages with OCR."""
    image_ocr_results = {}
    
    for page_idx, page in enumerate(pdf_response.pages):
        for i, img in enumerate(page.images):
            base64_data_url = img.image_base64
            
            def run_ocr_and_parse():
                # Step 1: basic OCR
                image_response = mistral_client.ocr.process(
                    document=ImageURLChunk(image_url=base64_data_url),
                    model="mistral-ocr-latest"
                )
                image_ocr_markdown = image_response.pages[0].markdown
                
                # Step 2: structure the OCR markdown
                structured = mistral_client.chat.parse(
                    model="pixtral-12b-latest",
                    messages=[
                        {
                            "role": "user",
                            "content": [
                                ImageURLChunk(image_url=base64_data_url),
                                TextChunk(text=(
                                    f"This is the image's OCR in markdown:\n{image_ocr_markdown}\n. "
                                    "Convert this into a structured JSON response with the OCR contents in a sensible dictionary."
                                ))
                            ]
                        }
                    ],
                    response_format=StructuredOCR,
                    temperature=0
                )
                
                structured_data = structured.choices[0].message.parsed
                pretty_text = json.dumps(structured_data.ocr_contents, indent=2, ensure_ascii=False)
                return pretty_text
            
            try:
                result = retry_with_backoff(run_ocr_and_parse, retries=4)
                image_ocr_results[(page_idx, img.id)] = result
            except Exception as e:
                print(f"❌ Failed at page {page_idx+1}, image {i+1}: {e}")
    
    # Reorganize results by page
    ocr_by_page = {}
    for (page_idx, img_id), ocr_text in image_ocr_results.items():
        ocr_by_page.setdefault(page_idx, {})[img_id] = ocr_text
    
    return ocr_by_page

## 5. Translation Functions
Create dedicated functions for translation:

In [6]:
# Translation system prompt
TRANSLATION_SYSTEM_INSTRUCTION = """
你是一位專業的技術文件翻譯者。請將我提供的英文 Markdown 內容翻譯成**台灣繁體中文**。

**核心要求：**
1.  **翻譯所有英文文字：** 你的主要工作是翻譯內容中的英文敘述性文字（段落、列表、表格等）。
2.  **保持結構與程式碼不變：**
    * **不要**更改任何 Markdown 標記（如 `#`, `*`, `-`, `[]()`, `![]()`, ``` ```, ` `` `, `---`）。
    * **不要**翻譯或修改程式碼區塊 (``` ... ```) 和行內程式碼 (`code`) 裡的任何內容。
    * 若有 JSON，**不要**更改鍵（key），僅翻譯字串值（value）。
3.  **處理專有名詞：** 對於普遍接受的英文技術術語、縮寫或專有名詞（例如 API, SDK, CPU, Google, Python 等），傾向於**保留英文原文**。但請確保翻譯了其他所有非術語的常規英文文字。
4.  **直接輸出結果：** 請直接回傳翻譯後的完整 Markdown 文件，不要添加任何額外說明。
"""

def translate_markdown_pages(pages, gemini_client):
    """Translate markdown pages using Gemini API."""
    translated_pages = []
    
    for idx, page in enumerate(pages):
        try:
            print(f"🔁 正在翻譯第 {idx+1} 頁...")
            
            response = gemini_client.models.generate_content(
                model="gemini-2.0-flash",
                config=types.GenerateContentConfig(
                    system_instruction=TRANSLATION_SYSTEM_INSTRUCTION
                ),
                contents=page
            )
            
            translated_md = response.text.strip()
            translated_pages.append(translated_md)
            
        except Exception as e:
            print(f"⚠️ 翻譯第 {idx+1} 頁失敗：{e}")
            translated_pages.append(page)
    
    return translated_pages

## 6. Main Processing Function
Create a main function that orchestrates the whole process:

In [7]:
def process_pdf_to_markdown(mistral_client, gemini_client):
    """Main function to process PDF to markdown with translation."""
    # Select and process PDF
    pdf_file = select_pdf_file()
    filename_stem = pdf_file.stem
    
    # Process PDF with OCR
    print("🔍 Processing PDF with OCR...")
    pdf_response = process_pdf_with_mistral_ocr(pdf_file, mistral_client)
    
    # Process images with OCR
    print("🖼️ Processing images with OCR...")
    ocr_by_page = process_images_with_ocr(pdf_response, mistral_client)
    
    # Create markdown pages
    markdown_pages = []
    image_folder_name = f"images_{filename_stem}"
    
    print("📝 Creating markdown pages with images...")
    for page_idx, page in enumerate(pdf_response.pages):
        images_dict = {img.id: img.image_base64 for img in page.images}
        
        md = page.markdown
        md = insert_ocr_below_images(md, ocr_by_page, page_idx)
        md = save_images_and_replace_links(md, images_dict, page_idx, image_folder=image_folder_name)
        
        markdown_pages.append(md)
    
    # Translate markdown pages
    print("🔄 Translating markdown pages...")
    translated_markdown_pages = translate_markdown_pages(markdown_pages, gemini_client)
    
    # Combine pages into complete markdown
    final_markdown_translated = "\n\n---\n\n".join(translated_markdown_pages)
    final_markdown_original = "\n\n---\n\n".join(markdown_pages)
    
    # Save files
    translated_md_name = f"{filename_stem}_translated.md"
    original_md_name = f"{filename_stem}_original.md"
    
    with open(translated_md_name, "w", encoding="utf-8") as f:
        f.write(final_markdown_translated)
    
    with open(original_md_name, "w", encoding="utf-8") as f:
        f.write(final_markdown_original)
    
    print(f"✅ 已儲存翻譯版：{translated_md_name}")
    print(f"✅ 已儲存原始英文版：{original_md_name}")
    print(f"✅ 圖片資料夾：{image_folder_name}")
    
    # Display translated markdown
    display(Markdown(final_markdown_translated))
    
    return {
        "translated_file": translated_md_name,
        "original_file": original_md_name,
        "image_folder": image_folder_name
    }

## 7. Execution Cell
Finally, add a simple execution cell:

In [8]:
if __name__ == "__main__":
    process_pdf_to_markdown(mistral_client, gemini_client)

📚 可用的 PDF 檔案：
  [1] paper3.pdf
  [2] paper4.pdf
✅ 已選擇：paper3.pdf
🔍 Processing PDF with OCR...
🖼️ Processing images with OCR...
📝 Creating markdown pages with images...
🔄 Translating markdown pages...
🔁 正在翻譯第 1 頁...
🔁 正在翻譯第 2 頁...
🔁 正在翻譯第 3 頁...
🔁 正在翻譯第 4 頁...
🔁 正在翻譯第 5 頁...
🔁 正在翻譯第 6 頁...
🔁 正在翻譯第 7 頁...
🔁 正在翻譯第 8 頁...
✅ 已儲存翻譯版：paper3_translated.md
✅ 已儲存原始英文版：paper3_original.md
✅ 圖片資料夾：images_paper3


# 第 2 章

## 認知

人類認知代表著一個複雜的資訊處理系統，它透過多個專業神經迴路的協調運作，實現感知、推理和目標導向行為 [98]。這種認知架構透過心智狀態運作，這些狀態是學習和推理發生的基礎。跨不同抽象層次處理資訊以及適應新情境的卓越能力，是 LLM 代理的一個重要靈感來源 [27]。
認知系統表現出幾個基本的架構屬性，如圖 1.1 所示。首先，學習功能跨越不同的心智狀態空間：它可以全面地發生在額葉（支援執行控制和認知）和顳葉（負責語言、記憶和聽覺處理）中，或者如圖中不同研究層次所示，專注於特定方面以進行有針對性的改進。其次，推理以不同的模式出現：它可以遵循結構化的模板，用於系統性的問題解決，並由額葉中的邏輯推理和認知靈活性支援，或者以非結構化的形式出現，用於靈活思考，尤其是在決策和執行控制功能中。第三，該系統表現出卓越的適應性，透過經驗不斷更新其心智狀態，同時利用監督式回饋（如小腦中的自適應錯誤校正）和非監督式環境統計，這反映在圖中各種認知功能的不同探索階段 [99]。
這些認知過程由模組化組織支援，該組織由不同但相互關聯的組件組成，形成一個有凝聚力的系統 [100]。這些模組包括將原始感官數據轉換為有意義的表示的感知系統、提供儲存和檢索資訊基礎的記憶系統、支援未來情境模擬的世界模型、透過強化引導行為改進的獎勵訊號、調節注意力和資源分配的情緒系統、制定決策的推理系統，以及將決策轉化為環境互動的行動系統。
雖然人類認知透過由演化塑造的複雜神經架構來實現這些屬性，但 LLM 代理嘗試使用大規模神經模型和演算法技術來近似類似的功能。理解這種生物與人工的平行關係對於開發更具能力的代理至關重要 [101]，因為它突出了當前系統與人類認知相比的成就和局限性。在適應性、泛化能力和情境理解等領域仍然存在顯著差異。
在本節中，我們首先探索 Learning（學習），研究它在心智狀態中發生的空間以及它所服務的特定目標。隨後，我們研究 Reasoning（推理），分析結構化和非結構化方法，最後專門探索規劃能力作為一種特殊的推理行動。

### 2.1 Learning（學習）

Learning（學習）代表智慧代理透過經驗將知識轉化為其心智狀態中的知識的根本過程。這種轉變發生在不同的認知空間中，從跨越完整心智狀態的整體更新到特定認知組件的改進。學習的範圍涵蓋了服務於不同目標的卓越能力：增強感知理解、提高推理能力以及發展更豐富的世界理解。

---

![img-0.jpeg](images_paper3/page_2_img_1.png)

> 📄 Image OCR Result：

```json
{
  "Learning": {
    "Space": {
      "Full": {
        "content": "SFT [41], PEFT [42], RLHF [43], ReFT [44], Agentic Models [45]"
      },
      "Partial": {
        "content": "CoT [46], Voyager [47], Reflexion [48], ActRe [49], Generative Agents [50]"
      }
    },
    "Perception": {
      "content": "CLIP [51], LLaVA [52], CogVLM [53], Qwen-Audio [54], R1-Searcher [45], Search-R1 [55]"
    },
    "Objective": {
      "Reasoning": {
        "content": "SKY-32B [56], Open Thoughts [57], LIMO [58], STAR [59], ReST [60], OpenR [61], LLaMA-Berry [62], RAGEN [63], OpenRI [64]"
      },
      "World": {
        "content": "Inner Monologue [65], DESP [66], Self-refine [67], CRITIC [68], Reflexion [48], ExpeL [69]"
      }
    }
  },
  "Dynamic": {
    "content": "ReAct [70], MCoT [71], ToT [72], LATS [73], RAP [74], GoT [75], PoT [76], DoT [77]"
  },
  "Structured": {
    "Static": {
      "content": "Self-Consistency [78], Self-refine [67], PHP [79], Self-Verification [80], CoVe [81]"
    },
    "Domain": {
      "content": "MathPrompter [82], PedCoT [83], Physics Reasoner [84]"
    },
    "Prompt": {
      "content": "CoT [46], Step-Back, [85] Ask Me Anything [86], CoK [87], SEK [88]"
    }
  },
  "Unstructured": {
    "Reasoning": {
      "Model": {
        "content": "DeepSeek-R1 [89], Claude 3.7, Sonnet [9], g1 [90]"
      },
      "Implicit": {
        "content": "Quiet-STaR [91], Coconut [92]"
      }
    }
  },
  "Planning": {
    "content": "DEPS [66], ProgPrompt [93], ADaPT [94], ToT [72], RAP [74], TravelPlanner [95], PDDL [96], Mind2Web [97]"
  }
}
```

圖 2.1：認知系統的說明性分類法，包括學習和推理範例。

人類的學習透過大腦適應性神經網路，在多個空間和目標中運作。 大腦透過整合系統協調整個網路的學習：海馬體促進情節體驗的快速編碼，小腦支援監督式學習以實現精確的運動技能，基底核透過多巴胺獎勵訊號實現強化學習，而皮質區域促進非監督式模式提取 [99]。 在更集中的層面上，特定的神經迴路可以進行有針對性的適應，從而實現專業的技能發展和知識獲取。 這些系統在不同的時間尺度上協同工作，從即時反應到終身發展，同時受到注意力、情緒和社會環境等因素的影響 [27]。

雖然在架構上存在根本差異，但 LLM 代理在其心智狀態空間中實現了類似的學習過程。 在綜合層面，它們透過對大量資料集進行預訓練來獲得廣泛的知識，展示了一種非監督式學習的形式。 在更集中的層面上，它們透過參數更新機制（例如監督式微調和強化學習）來完善特定的能力。 獨特的是，它們還展示了上下文學習能力，透過利用上下文來適應新任務，而無需更改參數。

---

在其注意力範圍內：這種能力反映了人類工作記憶的某些方面，但運作機制卻截然不同。
人類和人工學習系統之間的比較，為開發更具能力、更具適應性的代理提供了寶貴的見解。人類學習在效率、情境化以及與情感系統的整合方面展現了顯著的特徵，而基於 LLM 的方法在處理大型資料集、表示形式化知識以及跨領域綜合資訊方面，則展現了獨特的能力。這些互補的優勢為研究提供了富有成效的方向。在我們探索學習的基礎時，首先檢視學習在精神狀態中發生的空間，然後分析驅動學習過程的具體目標。

表 2.1：不同狀態修改的學習方法摘要。- 表示主要影響，而 $\circ$ 表示次要或無直接影響。

| 方法 | 模型 | 感知 | 推理 | 記憶 | 獎勵 | 世界模型 |
| :-- | :--: | :--: | :--: | :--: | :--: | :--: |
| Voyager [47] | 0 | 0 | 0 | $\bullet$ | 0 | 0 |
| Generative Agents [50] | 0 | 0 | 0 | $\bullet$ | 0 | 0 |
| Learn-by-interact [102] | $\bullet$ | 0 | 0 | $\bullet$ | 0 | 0 |
| RAGEN [63] | $\bullet$ | 0 | $\bullet$ | 0 | $\bullet$ | 0 |
| DigiRL [103] | $\bullet$ | 0 | $\bullet$ | 0 | $\bullet$ | 0 |
| R1-Searcher [45] | $\bullet$ | $\bullet$ | $\bullet$ | 0 | $\bullet$ | 0 |
| RewardAgent [104] | $\bullet$ | 0 | 0 | 0 | $\bullet$ | 0 |
| Text2Reward [105] | 0 | 0 | 0 | 0 | $\bullet$ | 0 |
| ARAMP [106] | $\bullet$ | 0 | 0 | 0 | $\bullet$ | 0 |
| ActRe [49] | $\bullet$ | 0 | $\bullet$ | 0 | 0 | $\bullet$ |
| WebDreamer [107] | 0 | 0 | 0 | 0 | 0 | $\bullet$ |
| RAP [74] | 0 | 0 | 0 | 0 | 0 | $\bullet$ |
| AutoManual [108] | 0 | 0 | 0 | $\bullet$ | 0 | $\bullet$ |

# 2.1.1 學習空間

LLM 代理中的學習方法代表了一種結構化的、資料驅動的範式，這與人類觀察到的探索性的、情感驅動的學習形成對比。雖然人類學習通常涉及積極的好奇心、動機和情感強化，但基於 LLM 的代理通常透過更正規化的過程來學習，例如訓練期間的參數更新或探索期間的結構化記憶形成。目前的代理架構試圖透過實施模擬人類學習某些方面的機制，同時利用計算系統的優勢來彌合這一差距。
智能代理中的學習發生在不同的空間中，包括底層模型 $\theta$ 和精神狀態 $M$，其中前者從根本上支持後者的能力和限制。形式上，我們將智能代理的內部狀態定義為一個元組 $\mathcal{I}=(\theta, M)$，它包括模型參數和精神狀態組成部分。正如我們在 1.2 中所闡述的，精神狀態可以進一步分解為不同的結構：

$$
M=\left\{M^{\text {mem }}, M^{\text {wm }}, M^{\text {emo }}, M^{\text {goal }}, M^{\text {rew }}\right\}
$$

其中 $M^{\text {mem }}$ 代表記憶，$M^{\text {wm }}$ 表示世界模型，$M^{\text {emo }}$ 表示情緒狀態，$M^{\text {goal }}$ 表示目標，$M^{\text {rew }}$ 表示獎勵訊號。
對底層模型的修改可以被視為完整的精神狀態學習，因為它們從根本上改變了代理的能力。雖然模型層級的修改可能在不同程度上影響不同的精神狀態，但對模型上下文視窗或外部結構的變更往往側重於特定的精神狀態組成部分。例如，來自環境的學習經驗和技能主要影響記憶，而利用 LLM 固有的預測能力可以增強世界模型。

完整精神狀態學習。 完整精神狀態學習透過全面修改底層模型 $\theta$ 來增強代理的能力，進而影響精神狀態 $M$ 的所有組成部分。這個過程從預訓練開始，它透過獲取大量的世界知識來建立語言模型的基礎，類似於人類嬰兒在發展過程中吸收環境資訊的方式，儘管這種方式更結構化、更廣泛。
後訓練技術代表了推進代理能力的基石。與人類大腦受到教育的塑造方式類似，這些技術在影響整個模型的同時，可以強調認知的不同方面。

---

開發。具體來說，各種形式的基於調整的學習使代理人能夠獲得特定領域的知識和邏輯推理能力。監督式微調 (SFT) [41] 是基礎方法，模型從人類標記的範例中學習，直接將知識編碼到模型的權重中。為了提高計算效率，出現了參數高效微調 (PEFT) 方法。Adapter-BERT [42] 引入了模組化設計，使模型能夠適應下游任務，而無需修改所有參數，而低秩適應 (LoRA) [109] 透過將權重更新分解為低秩矩陣來實現類似的結果，僅調整一小部分有效參數。

一些代理人能力與它們與人類偏好的一致程度密切相關，基於對齊的學習方法修改模型以重塑代理人底層表示的各個方面。從人類回饋中進行強化學習 (RLHF) [110] 透過在比較判斷中訓練獎勵模型並使用它來引導策略優化，使模型與人類價值觀保持一致。InstructGPT [43] 展示了這種方法如何顯著提高跨各種任務與使用者意圖的一致性。直接偏好優化 (DPO) [111] 透過將其重新制定為沒有明確獎勵建模的直接偏好學習，進一步簡化了此過程，在降低計算複雜度的同時保持了對齊品質。
強化學習 (RL) 為特定環境中的專業學習提供了一條有希望的途徑。RL 在增強推理能力方面表現出了特別的希望，基本上使代理人的底層模型能夠在思維空間內學習。諸如強化微調 (ReFT) [44] 等基礎性工作透過在線上強化學習獎勵下使用自動採樣的推理路徑進行微調來增強推理能力。DeepSeek-R1 [89] 透過基於規則的獎勵和群組相對策略優化 (GRPO) [112] 來推進此方法，而 Kimi k1.5 [113] 將上下文強化學習與最佳化的思維鏈技術相結合，以提高規劃流程和推理效率。在特定環境中，修改模型以增強代理人對動作和外部環境的理解已被證明是有效的，正如 DigiRL [103] 所展示的那樣，它實現了一種兩階段強化學習方法，使代理人能夠在真實世界的 Android 裝置模擬器上執行各種命令。

最近的研究試圖將代理人的動作空間直接整合到模型訓練中 [45, 55]，從而能夠透過 RL 或 SFT 方法學習不同狀態的適當動作。這種整合從根本上影響了代理人的記憶、獎勵理解和世界模型理解，為代理模型的出現指明了一個有希望的方向。

部分心理狀態學習 雖然透過模型修改進行完整的心理狀態學習提供了全面的能力更新，但學習專注於代理人心理狀態 $M$ 的特定組成部分代表了另一種必不可少且通常更有效的方法。這種部分心理狀態學習可以透過有針對性的模型更新或透過上下文適應而無需參數更改來實現。
上下文學習 (ICL) 說明了代理人如何在不修改整個模型的情況下有效地修改特定心理狀態組成部分。這種機制允許代理人透過利用其上下文視窗中的範例或指令來適應新任務，類似於人類工作記憶在快速任務適應中的作用。思維鏈 (CoT) [46] 展示了這種方法的有效性，展示了代理人如何在保持其基本模型參數不變的情況下增強特定的認知能力。
部分心理狀態學習的可行性已透過針對不同組成部分（例如記憶 $\left(M^{\text {meyn }}\right)$、獎勵 $\left(M^{\text {error }}\right)$ 和世界模型 $\left(M^{\text {wow }}\right)$）的各種方法得到證明。透過正常的溝通和社交互動，生成式代理人 [50] 展示了代理人如何累積和重播記憶，提取高階洞察力以指導動態行為規劃。在環境互動場景中，Voyager [47] 展示了代理人如何透過與 Minecraft 環境的直接互動來持續更新其技能庫，從而累積程序性知識而無需模型重新訓練。Learn-by-Interact [102] 透過合成直接環境互動的經驗數據進一步擴展了這種方法，從而消除了對手動註釋或強化學習框架的需求。此外，代理人可以從他們的錯誤中學習並透過反思來改進，正如 Reflexion [48] 所展示的那樣，它透過從重複的試錯經驗中獲得文本回饋來指導代理人未來的思考和行動。
對獎勵和世界模型的修改提供了部分心理狀態學習的另一個範例。ARMAP [106] 透過從代理人動作軌跡中提取獎勵模型來完善環境獎勵模型，為進一步學習奠定基礎。AutoMC [114] 透過環境探索構建密集的獎勵模型以支持代理人行為。同時，[107] 明確地利用 LLMs 作為世界模型來預測未來行動的影響，從而有效地修改了代理人對世界的理解 $\left(M^{\text {wow }}\right)$。ActRe [49] 以語言模型固有的世界理解為基礎，從軌跡構建任務，透過迭代訓練來增強代理人作為世界模型和推理引擎的能力。

---

# 2.1.2 學習目標

智慧代理的學習過程體現在它們與環境互動的各個層面。在輸入層面，代理學習更好地感知和解析環境訊息；在處理層面，代理學習如何基於現有知識或推理能力進行有效的推理；在理解層面，代理透過持續的互動來形成和優化它們對世界的理解。這種多層次的學習目標框架使代理能夠在不同的維度上持續進化，使它們能夠更好地處理複雜和動態的任務環境。

學習以獲得更好的感知 有效感知和處理來自環境的訊息的能力是代理智慧的基礎。為了增強感知能力，代理採用兩種主要的學習方法：擴展多模態感知和利用檢索機制。
多模態感知學習使代理能夠處理和整合多樣化的感官輸入，類似於人類的多感官整合，但不受生物學限制。這種能力透過像 CLIP [51] 這樣的進步得到了顯著發展，CLIP 開創了在共享嵌入空間中視覺和語言表徵的對齊。在此基礎上，像 LLaVA [52] 這樣的模型透過在圖像-文字對上訓練專用的投影器來增強視覺感知，而 CogVLM [53] 透過統一的表徵架構來推進視覺推理。

感知模態的擴展在多個感官領域持續進行。在音訊處理方面，QwenAudio [54] 展示了對多種聲學訊息（從語音到環境聲音）的統一編碼。 [115] 最近的工作甚至冒險進入觸覺感知領域，開發了對齊觸覺、視覺和語言表徵的數據集。這些進步使代理能夠更全面地與物理和數位環境互動。
代理還學習透過檢索機制來增強其觀察能力。與受限於即時感官輸入的人類感知不同，代理可以學習存取和整合來自廣泛外部知識儲存庫的訊息。像 RAG [116] 這樣的檢索增強方法透過將即時觀察與相關的儲存知識連接起來，從而增強感知理解。
基於檢索的代理的最新工作展示了增強主動訊息獲取能力的潛力。 Search-o1 [117] 引導推理模型透過提示學習主動檢索，從而擴展其知識邊界。更進一步，R1-Searcher [45] 和 Search-R1 [55] 將檢索能力直接整合到模型中，從而在推理過程中實現自主訊息檢索。這些進步表明了改善代理感知的一個有希望的方向：增強模型層級的主動感知能力，以豐富決策的基礎。這種方法可能代表未來代理發展的重要途徑。

學習以獲得更好的推理 推理是代理心理狀態與其行動之間的關鍵橋樑，使有效推理的能力和推理能力的發展對於智慧代理至關重要。現代代理中推理的基礎源於兩個關鍵要素：嵌入在其底層模型中的豐富世界知識，以及內部或透過上下文結構支援的強大的邏輯框架。這使得學習以獲得更好的推理成為代理開發中的重要目標。
推理能力的發展透過幾個關鍵現象得到證明。首先，高品質的推理數據直接增強了模型的推理能力；其次，這種高品質的數據通常需要驗證或獎勵模型才能進行有效的策展；第三，對基礎模型進行直接的強化學習可以自發地展現推理能力。
在 o1 系列發布後，推理在代理開發中的重要性再次被強調。一種常見的方法是從開源/閉源推理模型中收集和提煉數據。例如，SKY-32B [56] 從 QWQ-32B [118] 中提煉數據，以 450 美元的成本訓練了一個 32B 的推理模型。同樣，Open Thoughts [57] 透過從 R1 中提煉和合成數據集，以低成本訓練了 Bespoke-Stratos-32B。這些研究表明，即使沒有複雜的演算法設計，使用推理數據對基礎模型執行監督式微調 (Supervised Fine-Tuning, SFT) 也能有效地激活推理能力。
關於數據品質的另一個關鍵見解是，高度結構化的推理數據更有效地使代理和語言模型能夠學習推理過程。值得注意的是，LIMO [58] 證明了可以透過為複雜的推理任務構建長而有效的推理鏈，以極少的數據樣本構建強大的推理模型。這個見解源於他們的觀察，即語言模型本質上擁有足夠的推理知識，但需要高品質的推理路徑來激活這些能力。Li 等人的研究支持了這種觀點。

---

[119] 揭示了 Long CoT 和 Short CoT 從根本上教導模型學習推理結構，而不是特定內容，這表明自動選擇高品質推理數據可能成為未來的重要方向。

一種可行的探索方法是首先進行廣泛搜索，然後使用可驗證的環境或可訓練的獎勵模型來提供關於推理軌跡的回饋，從而過濾掉高品質的推理數據。 這種方法催生了幾個技術系列，它們利用不同的回饋機制來提高推理能力。

第一類遵循 STaR [59] 及其變體所例示的引導範例，這些變體實施了模型生成逐步理由，並通過在成功的推理路徑上進行微調來迭代改進的技術。 該系列包括 Quiet-STaR [91]、V-STaR [120] 和 rStar-Math [121]，後者通過強化學習原則專門增強數學推理。 通過迭代選擇正確的推理路徑進行訓練，這些方法通過連續的完善週期實現自我改進。
第二類通過更明確地結合強化學習原則來擴展這個範例。 ReST 系列，從引入強化自我訓練的原始 ReST [60] 開始，每個樣本執行多次嘗試（通常為 10 次），並從成功的推理實例中創建新的訓練數據集。 ReST-EM [122] 使用期望最大化來增強該方法，而 ReST-MCTS [122] 進一步整合了 Monte Carlo Tree Search，從而通過更複雜的探索策略實現了改進的推理能力。
幾種方法引入了策略獎勵模型 (Policy Reward Models, PRMs) 來提供關於推理路徑的品質回饋。 諸如 OpenR [61] 和 LLaMA-Berry [62] 之類的方法將推理任務建模為馬可夫決策過程 (Markov Decision Processes, MDPs)，並利用樹搜索來探索不同的推理路徑，同時使用 PRM 進行品質評估。 在特定領域的應用中，諸如 rStar-Math [121] 和 DeepSeekMath [112] 之類的方法已證明通過多輪自我迭代和平衡的探索-利用策略，在數學問題解決方面取得成功。 對於程式碼生成，o1-Coder [123] 利用 MCTS 生成具有推理過程的程式碼，而 Marco-o1 [123] 將此方法擴展到開放式任務。 這些實現突出了 MCTS 和 PRM 之間的協同作用如何實現有效的推理路徑探索，同時通過細粒度的監督來維持解決方案品質。
除了數據驅動方法之外，強化學習 (Reinforcement Learning, RL) 在增強語言模型的推理能力方面也取得了顯著的成功，DeepSeek R1 [89] 和 Kimi-K-1.5 [113] 等最新突破就證明了這一點。 用於 LLM 的 RL 的基礎可以追溯到幾個開創性的框架：ReFT [44] 引入了監督式微調和線上強化學習的組合，而 VeRL [124] 建立了一個開源框架，支持各種 RL 演算法，適用於高達 70B 參數的大型模型。 RFT [125] 進一步證明了獎勵引導優化在特定推理任務中的有效性。
在這些基礎上，後續的研究探索了多樣化的應用和改進。 OpenR1 [64] 和 RAGEN [63] 擴展了 RL 技術以增強通用推理能力，而 SWE-Gym [126] 等專門的實現證明了在軟體工程任務中的成功。 值得注意的是，DigiRL [103] 引入了用於數位世界代理增強的新方法。
最近的進展進一步將 RL 與工具使用和推理相結合。 Qwen-QwQ-32B [118] 採用強化學習和通用獎勵機制將工具調用納入推理過程中，從而可以在推理過程中無縫使用任意工具，並直接在模型內實現類似代理的能力。 類似地，RAGEN [63] 專注於多步驟代理場景，建立了一個用於複雜環境中代理強化學習的框架。 這些發展表明模型訓練和代理開發之間的融合日益增強，可能導致更加整合和有能力的智慧系統。 這些實現突出了 RL 如何有效地提高模型性能，同時減少對大規模註釋數據集的依賴，尤其是在複雜的推理場景中。

世界理解學習 代理智慧的一個關鍵方面是通過直接互動和經驗積累來理解世界如何運作的能力。 這種理解包括環境如何響應不同的行為以及這些行為帶來的後果。 通過與環境的持續互動，代理可以建立和完善他們的記憶、獎勵理解和世界模型，從成功和失敗中學習，從而更全面地掌握其操作領域。
最近的研究揭示了用於世界理解的體驗式學習的多種方法。 在基礎層面，Inner Monologue [65] 演示了代理如何通過持續的互動來積累基本的環境知識。 類似地，Learn-by-Interact [102] 表明有意義的理解可以從直接的環境參與中產生，而無需明確的獎勵機制。 更複雜的方法以 Minecraft 環境中的 DESP [66] 和 Voyager [47] 為例，其中代理不僅收集經驗，而且還積極處理它們：DESP 通過結果分析，Voyager 通過動態技能庫擴展。

---

透過先進的框架，對累積經驗的處理和利用已進一步系統化。Generative Agents [50] 引入了複雜的記憶重播機制，使代理程式能夠從過去的互動中提取高層次的見解。Self-refine [67] 和 Critic [68] 增強了這種系統化的方法，它們實施了經驗評估和改進的結構化週期。

通過環境互動優化對獎勵的理解，已成為世界理解的另一個關鍵面向。Text2Reward [105] 展示了代理程式如何透過人類的回饋，持續改進獎勵函數，使其更好地與任務目標和環境特徵對齊。同樣地，AutoManual [108] 透過持續的互動建立行為準則，開發經過獎勵驗證的協議，為理解環境獎勵和決策奠定基礎。這些基於互動的優化機制使代理程式能夠更好地理解環境動態，並產生更精確的獎勵信號，最終增強它們在複雜、動態環境中的適應性和決策能力。

在這些基礎之上，RAP [74] 將推理概念化為具有世界模型的規劃，代表了一項重大進展。透過將 LLM 重新用作推理代理程式和世界模型，RAP 使代理程式能夠在執行潛在行動之前模擬其結果，從而透過蒙地卡羅樹搜尋促進更有效的規劃。這種方法允許代理程式在探索和利用之間取得適當的平衡，策略性地探索推理空間。

利用世界模型進行代理程式學習的進一步創新包括 ActRe [127]，它顛倒了典型的推理-行動序列，首先執行行動，然後生成事後解釋。這種合理化行動的能力展示了 LLM 對世界動態的內在理解，從而實現自主軌跡註釋並促進對比自我訓練。

[128] 強調了認知地圖在世界理解中的重要性，他們表明，受人類認知啟發的結構化心智表徵顯著增強了 LLM 在新環境中的外推能力。這些認知地圖不僅改善了規劃，還展現了類似人類的特徵，例如結構化的心智模擬和快速適應。

在基於網路的環境中，[107] 和 [129] 的最新研究表明，LLM 可以作為有效的世界模型，來預測網路互動的結果。透過在執行行動之前模擬潛在的狀態變化，這些方法能夠實現更安全、更有效的決策，尤其是在行動可能是不可逆轉的環境中。

透過像 Reflexion [48] 和 ExpeL [69] 這樣的系統，代理程式透過自主管理經驗收集、分析和應用的完整週期，改進了經驗學習，使它們能夠有效地從成功和失敗中學習。

這些發展共同說明了世界模型如何越來越成為代理程式學習系統的核心，為理解環境動態奠定基礎，並使在複雜、互動的環境中進行更有效的規劃、推理和決策成為可能。

# 2.2 Reasoning （推理）

Reasoning（推理）代表智能行為的關鍵，將原始資訊轉化為可執行的知識，從而推動問題解決和決策。對於人類和人工代理程式來說，它能夠進行邏輯推理、假設生成以及與世界的有目的互動。在人類認知中，推理是透過多種策略產生的：演繹推理將一般規則應用於特定案例，歸納推理從特定實例建立概括，而溯因推理從不完整的數據構建合理的解釋 [130, 131]。這些過程透過啟發法（簡化不確定情況下決策的心智捷徑）得到增強，並透過環境回饋不斷改進，確保推理紮根於現實並適應變化。

對於基於 LLM 的代理程式來說，推理扮演著類似的角色，將它們從反應式系統提升為能夠進行複雜認知的積極實體。透過推理，這些代理程式處理多模態輸入，整合不同的知識來源，並制定連貫的策略以實現目標。環境發揮雙重作用：提供為推理提供燃料的資訊，並作為測試推理行動的試驗場，創建一個回饋迴路，使代理程式能夠驗證推論並從錯誤中學習。

在基於 LLM 的代理程式中，推理可以正式定義為基於心智狀態的行動選擇過程，代表著感知和行動之間的關鍵橋樑。更準確地說，給定時間 t 的心智狀態 Mt，推理可以形式化為函數 $\mathrm{R}(\mathrm{Mt}) \rightarrow$ at，其中 at 代表所選的行動。這個過程跨多個

---

![img-1.jpeg](images_paper3/page_8_img_1.png)

> 📄 Image OCR Result：

```json
{
  "Structured Reasoning": {
    "Static": "input -> [node1] -> [node2] -> [node3] -> output",
    "Dynamic": "input -> [node1] -> [node2] -> [node3] -> output"
  },
  "Unstructured Reasoning": {
    "Explicit": "input -> [language space] -> output",
    "Implicit": "input -> [language space] -> [language space] -> output"
  },
  "Planning": "input -> [reasoning node] -> [planned node] -> [reasoning node] -> output"
}
```

圖 2.2：基於 LLM 的代理中推理範例的比較。
各種環境 - 文字、數位和物理世界 - 在這些環境中，完成任務通常需要單一推理步驟或多個推理動作的組合。

推理動作的組合自然會產生兩種不同的方法：結構化推理和非結構化推理。結構化推理 $\left(R_{s}\right)$ 可以形式化為顯式組合 $R_{s}=R_{1} \circ R_{2} \circ \ldots \circ R_{n}$，其中每個 $R_{i}$ 代表一個具有清晰邏輯依賴性的離散推理步驟。相反地，非結構化推理 $\left(R_{u}\right)$ 採用更全面的形式 $R_{u}=f\left(M_{t}\right)$，其中組合保持隱含且靈活，允許動態適應上下文。這種雙重框架反映了人類的認知，其中結構化推理與我們顯式的邏輯推導過程相似，而非結構化推理則反映了我們直觀解決問題和模式識別的能力。

環境在這種形式化中扮演著至關重要的角色，既是影響心智狀態更新的觀察結果 $o_{t}$ 的來源 $\left(M_{t}=L\left(M_{t-1}, a_{t-1}, o_{t}\right)\right)$，也是推理結果的試驗場。這創建了一個連續的回饋迴路，其中推理不僅驅動了動作選擇，而且還影響了代理的心智狀態如何演變，從而能夠透過經驗迭代改進推理策略。
在本節中，我們將檢驗這些推理方法如何在實踐中體現出來。我們先從結構化推理開始，它強調系統的問題分解和多步驟邏輯鏈。然後，我們將探索非結構化推理，它允許靈活的響應模式和平行解決方案探索。最後，我們將研究計畫作為一種特殊的推理形式，它結合了結構化和非結構化方法來解決複雜的、長期的任務。

# 2.2.1 結構化推理

結構化推理代表一種有條理的問題解決方法，它採用顯式的組織框架來指導推理過程。與非結構化方法不同，結構化推理使推理步驟的組合變得明確，可以形式化為 $R_{s}=R_{1} \circ R_{2} \circ \ldots \circ R_{n}$，其中每個 $R_{i}$ 代表一個具有清晰邏輯依賴性的離散推理步驟。在這種公式中，每個推理節點都是一個顯式執行的計算單元，節點之間的連接代表明確的資訊流路徑。這種方法能夠更系統地探索解決方案空間，並透過有意的逐步分析來促進更穩健的決策，從而在整個推理過程中提供高度的可解釋性和可追溯性。

### 2.2.1.1 動態推理結構

動態推理結構允許在解決問題的過程中自適應地建構推理路徑，從而建立可以根據中間結果和見解進行調整的多功能框架。
線性循序推理 線性結構將推理構建為一系列循序步驟，其中每個步驟都建立在前一個步驟的基礎上。ReAct [70] 透過以交替方式將推理追蹤與特定任務動作結合起來，來說明了這一點。這種組合允許推理追蹤引導和修改動作計畫，而動作可以存取外部來源以獲取更多資訊。這種相互作用提高了推理的完整性和環境適應性。